<a href="https://colab.research.google.com/github/AzucenaMV/CapstoneProject/blob/master/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import datetime
from functools import partial, reduce  

In [0]:
def sampling(dfo, freq = 'D',interpolate = False, method = 'linear'):
  df = dfo.copy()
  df.index = pd.to_datetime(df['date'],infer_datetime_format=True)
  del df['date']
  df_interpol = df.resample(freq).mean()
  if interpolate:
    df_interpol[df.columns] = df_interpol[df.columns].interpolate(method=method)
  return df_interpol

def date_range(dfo, start = '31-12-2009', end = '31-12-2019'):
  end = datetime.datetime.strptime(end, "%d-%m-%Y")
  start = datetime.datetime.strptime(start, "%d-%m-%Y")
  df=dfo.copy()
  df=df[(df.date >= start) & (df.date <= end)]
  return df

def get_data(url, verbose = False):
  raw_url = url.replace("https://github.com/AzucenaMV/CapstoneProject/tree/master/","https://raw.githubusercontent.com/AzucenaMV/CapstoneProject/master/")
  data_dict = {}
  data = pd.read_html(url)[0]
  csv_names = data.Name[data.Name.str.contains(".csv")]
  for name in csv_names:
    dict_name = name.replace(".csv","")
    dict_name = dict_name[0].lower() + dict_name[1:]
    if verbose:
      print(name)
    frame = pd.read_csv(raw_url+name,names = ['date',dict_name],skiprows=1, na_values = ".")
    frame['date'] = pd.to_datetime(frame['date'],infer_datetime_format=True)
    frame = date_range(frame)
    data_dict[dict_name] = sampling(frame)
  return data_dict

# Data Collection


##Covariates

In [0]:
url_dict = {}
url_dict['chem'] = "https://github.com/AzucenaMV/CapstoneProject/tree/master/data/chem/dateVar/"
url_dict['oil'] = "https://github.com/AzucenaMV/CapstoneProject/tree/master/data/oil/dateVar/"
url_dict['air'] = "https://github.com/AzucenaMV/CapstoneProject/tree/master/data/airline/dateVar/"
url_dict['econ'] = "https://github.com/AzucenaMV/CapstoneProject/tree/master/data/economics/"
url_dict['others'] = "https://github.com/AzucenaMV/CapstoneProject/tree/master/data/others/dateVar/"

In [0]:
# Reading data from links
data_dict = {}
for name,url in url_dict.items():                                                        
  data = get_data(url)
  reduce_df = partial(pd.merge, left_index=True, right_index=True, how = 'outer')                                                              
  data_dict[name] = reduce(reduce_df, data.values())  

In [0]:
# Removing data with less than n points
n = 39
category = 'others'
for col in data_dict[category].columns:
  if data_dict[category][col].notnull().sum() < n:
    del data_dict[category][col]

Fundamentals

In [0]:
#url_dict_fund = {}
#url_dict_fund['others'] = "https://github.com/AzucenaMV/CapstoneProject/tree/master/data/others/fundamentals/"

In [0]:
#columns = ['epsfxq','revtq','date']
#url = url_dict_fund['others']
#raw_url = url.replace("https://github.com/AzucenaMV/CapstoneProject/tree/master/","https://raw.githubusercontent.com/AzucenaMV/CapstoneProject/master/")
#data = pd.read_html(url)[0]
#txt_names = data.Name[data.Name.str.contains(".txt")]
#for name in txt_names:
#  dict_name = name.replace(".txt","")
#  frame = pd.read_csv(raw_url+name)
#  frame['date'] = pd.to_datetime(frame['datadate'],infer_datetime_format=True)
#  frame = frame[columns]
#  frame = date_range(frame)
#  print(frame)

In [0]:
# Reading data from links
data_dict_fund = {}
for name,url in url_dict.items():                                                        
  data = get_data(url)
  reduce_df = partial(pd.merge, left_index=True, right_index=True, how = 'outer')                                                              
  data_dict[name] = reduce(reduce_df, data.values())  

##Target Variable


In [19]:
# Get tickers from S&P 500 companies 
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
table = data[0]
table_sector = table[['Symbol','GICS Sector', 'GICS Sub Industry']]
table_sector.columns = ['tic', 'sector', 'sub_industry']
table_sector = table_sector.apply(lambda x: x.str.lower() if x.name in ['sub_industry','sector'] else x)

# Get stock price share
stock = pd.read_csv('https://raw.githubusercontent.com/AzucenaMV/CapstoneProject/master/data/sp500/wrds_sp500_stock_price_share.csv')
stock = stock[['datadate', 'tic', 'prccd', 'cshoc']]
stock.columns = ['date', 'tic', 'price', 'share']
stock['date'] = pd.to_datetime(stock['date'],format='%Y%M%d').dt.strftime('%Y-%M-%d')
stock['market_val'] = stock['price'] * stock['share']
stock = stock.merge(table_sector, on='tic')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# Splitting sectors
chemicals = {'specialty':['specialty chemicals'],'non-specialty':['fertilizers & agricultural chemicals','commodity chemicals','diversified chemicals']}
oil = {'upstream':['oil & gas exploration & production','oil & gas drilling'],'midstream':['oil & gas storage & transportation'],'downstream':['oil & gas refining & marketing'],'integrated':['integrated oil & gas'],'services':['oil & gas equipment & services']}
airline = {'legacy':['airlines']}
industry_dicts = [chemicals,oil,airline]

dic_sub_industry_df ={} # df for each sub industry
for dic in industry_dicts:
  for sub in dic.keys():
    dic_sub_industry_df[sub] = stock.loc[stock['sub_industry'].isin(dic[sub]), :]

In [0]:
temp = stock.groupby('date')['market_val', 'share'].agg('sum')
df_market = pd.DataFrame(temp['market_val'] / temp['share'], columns=['market']).reset_index()

In [0]:
# Aggregating by subindustry
df_price_agg = df_market
for sub, df in dic_sub_industry_df.items():
  temp = df.groupby('date')['market_val', 'share'].agg('sum')
  df_outcome = pd.DataFrame(temp['market_val'] / temp['share'], columns=[sub])
  df_price_agg = df_price_agg.merge(df_outcome, on='date', how='left') 

In [33]:
df_price_agg.head()

,market,specialty,non-specialty,upstream,midstream,downstream,integrated,services,legacy
date,,,,,,,,,
2009-12-31,33.250038,35.502226,61.699924,61.808103,24.678472,17.475954,70.332884,49.083616,12.023977
2010-01-04,33.763571,36.854109,63.452263,63.701038,25.282004,18.665005,71.686781,50.792623,11.839700
2010-01-05,33.871269,37.481396,64.457564,63.740866,25.449366,19.232099,72.044686,51.158975,12.638237
2010-01-06,33.862443,37.850241,66.455050,64.336719,26.090430,19.648771,72.457241,52.252925,12.423343
2010-01-07,34.003506,37.961343,66.449674,63.880724,25.908983,19.778143,72.281186,52.543386,12.782049


In [0]:
# Add target data to the data dictionary
data_dict['target'] = df_price_agg

In [0]:
# first calcualte the percentage change
# relative change w.r.t. the market
#df_price_agg.index = df_price_agg['date']
#del df_price_agg['date']
#df_relative_change = df_price_agg.pct_change().div(df_price_agg['market'].pct_change(), axis=0)
#df_relative_change

In [0]:
# get monthly data for stock price
#temp = df_relative_change.reset_index()
#temp['start_date'] = temp['date'].str[:7]+'-01'
#start_date = pd.DataFrame(temp.groupby('start_date')['date'].min()).reset_index()
#temp = start_date.merge(temp, on='date',how='inner')
#temp = temp.drop(columns=['date', 'start_date_y']).rename(columns={'start_date_x':'date'})
#temp['date'] = pd.to_datetime(temp['date'])
#df_relative_change_monthly = temp
#df_relative_change_monthly

In [0]:
#df_combined = df_others.reset_index().merge(df_relative_change_monthly, on='date').set_index('date').drop(columns=['market'])
#df_combined

In [0]:
#import seaborn as sns
#import numpy as np

#corr = df_combined.corr()
#sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
#            square=True, cbar=False)
#ax[0].set_title('Legacy Carriers')
#ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=45, horizontalalignment='right')

## Saving

In [0]:
import pickle

with open('data_dict.pickle', 'wb') as handle:
    pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
from google.colab import files
files.download('data_dict.pickle')